# FastGRNN and FastRNN in Keras

This is modified to Keras implementation of FastGRNN and FastRNN cell from a original simple notebook that illustrates the usage of tensorflow implementation. This is just only training result. The USPS dataset is used as with the original code.  
Original code is as follows.  
https://github.com/microsoft/EdgeML/tree/master/examples/tf/FastCells
  
Please refer to `fetch_usps.py` and run it for downloading and cleaning up the dataset.  

The original code license is as follows.  
Copyright (c) Microsoft Corporation. All rights reserved.  
Licensed under the MIT license.

In [6]:
import helpermethods
import tensorflow as tf
import numpy as np
import sys
import os
from tensorflow.keras.layers import Dense, Input, RNN
from tensorflow.keras.models import Model, Sequential, load_model
from tensorflow.keras import backend as K
from tensorflow.keras.optimizers import Adam

#Provide the GPU number to be used
os.environ['CUDA_VISIBLE_DEVICES'] =''

#FastRNN and FastGRNN imports
from rnn import FastGRNNCellKeras, FastRNNCellKeras

# Fixing seeds for reproducibility
tf.set_random_seed(42)
np.random.seed(42)

# USPS Data

--- Here is completely copied from the original code. ---  
  
  
It is assumed that the USPS data has already been downloaded and processed with [fetch_usps.py](fetch_usps.py) and [process_usps.py](process_usps.py), and is present in the `./usps10` subdirectory.

Note: Even though usps10 is not a time-series dataset, it can be assumed as, a time-series where each row is coming in at one single time.
So the number of timesteps = 16 and inputDims = 16

In [7]:
#Loading and Pre-processing dataset for FastCells
dataDir = "usps10"
(dataDimension, numClasses, Xtrain, Ytrain, Xtest, Ytest, mean, std) = helpermethods.preProcessData(dataDir)
print("Feature Dimension: ", dataDimension)
print("Num classes: ", numClasses)

Feature Dimension:  256
Num classes:  10


# Model Parameters

---Here is completely copied from the original code.---  
  

FastRNN and FastGRNN work for most of the hyper-parameters with which you could acheive decent accuracies on LSTM/GRU. Over and above that, you can use low-rank, sparsity and quatization to reduce model size upto 45x when compared to LSTM/GRU.

In [12]:
cell = "FastGRNN" # Choose between FastGRNN, FastRNN  ###, UGRNN, GRU and LSTM

inputDims = 16 #features taken in by RNN in one timestep
hiddenDims = 32 #hidden state of RNN

totalEpochs = 300
batchSize = 100

learningRate = 0.01
decayStep = 200
decayRate = 0.1

outFile = None #provide your file, if you need all the logging info in a file

#low-rank parameterisation for weight matrices. None => Full Rank
wRank = None 
uRank = None 

#Sparsity of the weight matrices. x => 100*x % are non-zeros
sW = 1.0 
sU = 1.0

#Non-linearities for the RNN architecture. Can choose from "tanh, sigmoid, relu, quantTanh, quantSigm"
update_non_linearity = "tanh"
gate_non_linearity = "sigmoid"

assert dataDimension % inputDims == 0, "Infeasible per step input, Timesteps have to be integer"

# FastCell Graph Object

In [13]:
#Create appropriate RNN cell object based on choice
if cell == "FastGRNN":
    FastCell = FastGRNNCellKeras(hiddenDims, gate_non_linearity=gate_non_linearity,
                            update_non_linearity=update_non_linearity,
                            wRank=wRank, uRank=uRank)
elif cell == "FastRNN":
    FastCell = FastRNNCellKeras(hiddenDims, update_non_linearity=update_non_linearity,
                           wRank=wRank, uRank=uRank)

    
###!!!Other Cells is not implemented for Keras!!!!###
elif cell == "UGRNN":
    FastCell = UGRNNLRCell(hiddenDims, update_non_linearity=update_non_linearity,
                           wRank=wRank, uRank=uRank)
elif cell == "GRU":
    FastCell = GRULRCell(hiddenDims, update_non_linearity=update_non_linearity,
                         wRank=wRank, uRank=uRank)
elif cell == "LSTM":
    FastCell = LSTMLRCell(hiddenDims, update_non_linearity=update_non_linearity,
                          wRank=wRank, uRank=uRank)
else:
    sys.exit('Exiting: No Such Cell as ' + cell)

# FastCell Trainer Model for Keras

In [14]:
x = inputs = Input(shape=[int(dataDimension / inputDims), inputDims], name='input')
layer_shape = K.int_shape(x)#x.get_shape()
print(layer_shape)
x = RNN(FastCell, return_sequences=False, name='rnn')(x)
out = Dense(numClasses, activation='softmax', name='dense')(x)
model = Model(inputs=inputs, outputs=out)
Adam_ = Adam(lr=0.01)
model.compile(optimizer=Adam_, loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()
#Y = tf.placeholder("float", [None, numClasses])

(None, 16, 16)
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           (None, 16, 16)            0         
_________________________________________________________________
rnn (RNN)                    (None, 32)                1602      
_________________________________________________________________
dense (Dense)                (None, 10)                330       
Total params: 1,932
Trainable params: 1,932
Non-trainable params: 0
_________________________________________________________________


# Training

In [15]:
Xtrain_ = np.reshape(Xtrain, [-1, int(dataDimension / inputDims), inputDims])
Xtest_ = np.reshape(Xtest, [-1, int(dataDimension / inputDims), inputDims])
model.fit(Xtrain_, Ytrain, batchSize, epochs=totalEpochs, validation_data=(Xtest_, Ytest))

Train on 7291 samples, validate on 2007 samples
Epoch 1/300
7291/7291 [==============================] - 1s 161us/sample - loss: 1.1139 - acc: 0.6160 - val_loss: 0.7114 - val_acc: 0.7444
Epoch 2/300
7291/7291 [==============================] - 1s 95us/sample - loss: 0.4190 - acc: 0.8615 - val_loss: 0.5000 - val_acc: 0.8530
Epoch 3/300
7291/7291 [==============================] - 1s 93us/sample - loss: 0.2875 - acc: 0.9087 - val_loss: 0.4577 - val_acc: 0.8690
Epoch 4/300
7291/7291 [==============================] - 1s 88us/sample - loss: 0.2236 - acc: 0.9285 - val_loss: 0.4128 - val_acc: 0.8869
Epoch 5/300
7291/7291 [==============================] - 1s 98us/sample - loss: 0.1772 - acc: 0.9450 - val_loss: 0.3775 - val_acc: 0.8924
Epoch 6/300
7291/7291 [==============================] - 1s 98us/sample - loss: 0.1540 - acc: 0.9493 - val_loss: 0.3665 - val_acc: 0.8979
Epoch 7/300
7291/7291 [==============================] - 1s 97us/sample - loss: 0.1326 - acc: 0.9589 - val_loss: 0.3381 - v

7291/7291 [==============================] - 1s 82us/sample - loss: 0.0302 - acc: 0.9892 - val_loss: 0.3617 - val_acc: 0.9228
Epoch 60/300
7291/7291 [==============================] - 1s 83us/sample - loss: 0.0179 - acc: 0.9952 - val_loss: 0.3699 - val_acc: 0.9233
Epoch 61/300
7291/7291 [==============================] - 1s 84us/sample - loss: 0.0134 - acc: 0.9966 - val_loss: 0.3708 - val_acc: 0.9268
Epoch 62/300
7291/7291 [==============================] - 1s 86us/sample - loss: 0.0083 - acc: 0.9989 - val_loss: 0.4009 - val_acc: 0.9253
Epoch 63/300
7291/7291 [==============================] - 1s 80us/sample - loss: 0.0458 - acc: 0.9855 - val_loss: 0.4052 - val_acc: 0.9148
Epoch 64/300
7291/7291 [==============================] - 1s 83us/sample - loss: 0.0492 - acc: 0.9834 - val_loss: 0.3565 - val_acc: 0.9268
Epoch 65/300
7291/7291 [==============================] - 1s 87us/sample - loss: 0.0360 - acc: 0.9879 - val_loss: 0.3509 - val_acc: 0.9273
Epoch 66/300
7291/7291 [================

Epoch 118/300
7291/7291 [==============================] - 1s 92us/sample - loss: 0.0085 - acc: 0.9978 - val_loss: 0.4129 - val_acc: 0.9228
Epoch 119/300
7291/7291 [==============================] - 1s 88us/sample - loss: 0.0122 - acc: 0.9966 - val_loss: 0.4091 - val_acc: 0.9218
Epoch 120/300
7291/7291 [==============================] - 1s 92us/sample - loss: 0.0301 - acc: 0.9893 - val_loss: 0.4073 - val_acc: 0.9278
Epoch 121/300
7291/7291 [==============================] - 1s 90us/sample - loss: 0.0250 - acc: 0.9915 - val_loss: 0.4043 - val_acc: 0.9302
Epoch 122/300
7291/7291 [==============================] - 1s 92us/sample - loss: 0.0152 - acc: 0.9953 - val_loss: 0.3975 - val_acc: 0.9283
Epoch 123/300
7291/7291 [==============================] - 1s 92us/sample - loss: 0.0095 - acc: 0.9979 - val_loss: 0.4178 - val_acc: 0.9327
Epoch 124/300
7291/7291 [==============================] - 1s 94us/sample - loss: 0.0088 - acc: 0.9975 - val_loss: 0.4010 - val_acc: 0.9302
Epoch 125/300
7291/7

7291/7291 [==============================] - 1s 90us/sample - loss: 0.0018 - acc: 0.9999 - val_loss: 0.4102 - val_acc: 0.9357
Epoch 177/300
7291/7291 [==============================] - 1s 87us/sample - loss: 0.0012 - acc: 0.9999 - val_loss: 0.4181 - val_acc: 0.9352
Epoch 178/300
7291/7291 [==============================] - 1s 88us/sample - loss: 0.0015 - acc: 0.9999 - val_loss: 0.4152 - val_acc: 0.9357
Epoch 179/300
7291/7291 [==============================] - 1s 95us/sample - loss: 0.0021 - acc: 0.9995 - val_loss: 0.4452 - val_acc: 0.9347
Epoch 180/300
7291/7291 [==============================] - 1s 87us/sample - loss: 0.0158 - acc: 0.9957 - val_loss: 0.4632 - val_acc: 0.9223
Epoch 181/300
7291/7291 [==============================] - 1s 90us/sample - loss: 0.0314 - acc: 0.9892 - val_loss: 0.4165 - val_acc: 0.9278
Epoch 182/300
7291/7291 [==============================] - 1s 91us/sample - loss: 0.0366 - acc: 0.9886 - val_loss: 0.4795 - val_acc: 0.9198
Epoch 183/300
7291/7291 [=========

7291/7291 [==============================] - 1s 87us/sample - loss: 0.0307 - acc: 0.9892 - val_loss: 0.3818 - val_acc: 0.9287
Epoch 235/300
7291/7291 [==============================] - 1s 95us/sample - loss: 0.0131 - acc: 0.9951 - val_loss: 0.4114 - val_acc: 0.9342
Epoch 236/300
7291/7291 [==============================] - 1s 88us/sample - loss: 0.0102 - acc: 0.9967 - val_loss: 0.4117 - val_acc: 0.9302
Epoch 237/300
7291/7291 [==============================] - 1s 89us/sample - loss: 0.0076 - acc: 0.9981 - val_loss: 0.4108 - val_acc: 0.9307
Epoch 238/300
7291/7291 [==============================] - 1s 88us/sample - loss: 0.0042 - acc: 0.9990 - val_loss: 0.4310 - val_acc: 0.9292
Epoch 239/300
7291/7291 [==============================] - 1s 90us/sample - loss: 0.0044 - acc: 0.9990 - val_loss: 0.4204 - val_acc: 0.9327
Epoch 240/300
7291/7291 [==============================] - 1s 88us/sample - loss: 0.0034 - acc: 0.9997 - val_loss: 0.4296 - val_acc: 0.9352
Epoch 241/300
7291/7291 [=========

7291/7291 [==============================] - 1s 123us/sample - loss: 0.0018 - acc: 0.9999 - val_loss: 0.4616 - val_acc: 0.9287
Epoch 293/300
7291/7291 [==============================] - 1s 112us/sample - loss: 0.0010 - acc: 0.9999 - val_loss: 0.4556 - val_acc: 0.9283
Epoch 294/300
7291/7291 [==============================] - 1s 113us/sample - loss: 9.5032e-04 - acc: 0.9999 - val_loss: 0.4604 - val_acc: 0.9283
Epoch 295/300
7291/7291 [==============================] - 1s 88us/sample - loss: 0.0023 - acc: 0.9995 - val_loss: 0.4537 - val_acc: 0.9297
Epoch 296/300
7291/7291 [==============================] - 1s 93us/sample - loss: 0.0015 - acc: 0.9999 - val_loss: 0.4556 - val_acc: 0.9322
Epoch 297/300
7291/7291 [==============================] - 1s 103us/sample - loss: 0.0027 - acc: 0.9996 - val_loss: 0.4643 - val_acc: 0.9287
Epoch 298/300
7291/7291 [==============================] - 1s 92us/sample - loss: 0.0022 - acc: 0.9997 - val_loss: 0.4532 - val_acc: 0.9283
Epoch 299/300
7291/7291 [=